<a href="https://colab.research.google.com/github/EricRibeiroAlves/DeteccaoFakeNews/blob/main/Modelos/%5BAP_T%5D_TI_Modelo_PAC_%2B_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Carregar os dados
dat_fake = "/content/drive/MyDrive/Eng. Controle e Automação/8º Semestre/AP/dataset_FakeNews/Fake.csv"
dat_real = "/content/drive/MyDrive/Eng. Controle e Automação/8º Semestre/AP/dataset_FakeNews/True.csv"
dt_fake = pd.read_csv(dat_fake)
dt_real = pd.read_csv(dat_real)

dt_fake['label'] = 0  # Fake news -> 0
dt_real['label'] = 1  # Real news -> 1

dt = pd.concat([dt_fake, dt_real], ignore_index=True)





In [ ]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    dt['text'], dt.label, test_size=0.2, random_state=7
)

# Transformar os textos em vetores TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# Treinar o PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train, y_train)

# Predições do PAC no conjunto de treino e teste
pac_train_preds = pac.decision_function(tfidf_train)  # Margem de confiança
pac_test_preds = pac.decision_function(tfidf_test)

# Combinar as predições do PAC com os vetores TF-IDF
train_combined = np.hstack((tfidf_train.toarray(), pac_train_preds.reshape(-1, 1)))
test_combined = np.hstack((tfidf_test.toarray(), pac_test_preds.reshape(-1, 1)))


In [ ]:
# Criar o modelo de Deep Learning
model = Sequential([
    Dense(128, activation='relu', input_shape=(train_combined.shape[1],)),  # Entrada combinada
    Dropout(0.3),  # Prevenção de overfitting
    Dense(64, activation='relu'),  # Camada intermediária
    Dropout(0.3),
    Dense(32, activation='relu'),  # Camada adicional
    Dense(1, activation='sigmoid')  # Saída binária
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(train_combined, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2,
                    verbose=1)

# Avaliar no conjunto de teste
loss, accuracy = model.evaluate(test_combined, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")